## [目标] 让机器学会理解语言的情感

* 其实每一个字对于机器来说都是没有意义的，要怎么让机器做到情感理解呢？
* 想办法把文字转成机器可以理解的方式
* 会带到以下观念：
    - 断词系统
    - 神奇的词项量
    - 建立一个语意分类模型

In [1]:
import pandas as pd
import numpy as np
import collections
import random
import warnings
import sys
import os
import multiprocessing
import matplotlib.pyplot as plt

## 深度学习框架 PyTorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

warnings.filterwarnings('ignore')

### 资料集介绍

> 语意的训练资料，也是需要标记的，许多网站服务上，会有使用者的评论以及给分或是打星，这些都是很棒的训练资料！因为可以很清楚了解到使用者在留言这句话的时候，是觉得很开心很满意，或是非常的不满。可以透过大数据课程中学到的爬虫技巧，抓取各式各样的模型训练资料！

### 因为时间有限，网路上也有很多热心人士准备好的研究资料，例如下面这些：
* 携程网顾客评论资料：
    * https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/ChnSentiCorp_htl_all/intro.ipynb
* 复旦大学文本分类语料库测试语料
    * http://www.nlpir.org/wordpress/2017/10/02/%e6%96%87%e6%9c%ac%e5%88%86%e7%b1%bb%e8%af%ad% e6%96%99%e5%ba%93%ef%bc%88%e5%a4%8d%e6%97%a6%ef%bc%89%e6%b5%8b%e8%af%95%e8% af%ad%e6%96%99/
* 电商平台商品描述语料
    * https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/online_shopping_10_cats/intro.ipynb

### 就以携程网的饭店论资料来试试吧！

In [2]:
res_df = pd.read_csv('./data/hotel_commients.csv')

In [3]:
## 看一下比较属正面的话

res_df.head(10)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
5,1,总的来说，这样的酒店配这样的价格还算可以，希望他赶快装修，给我的客人留些好的印象
6,1,价格比比较不错的酒店。这次免费升级了，感谢前台服务员。房子还好，地毯是新的，比上次的好些。早...
7,1,不错，在同等档次酒店中应该是值得推荐的！
8,1,入住丽晶，感觉很好。因为是新酒店，的确有淡淡的油漆味，房间内较新。房间大小合适，卫生间设备齐...
9,1,1。酒店比较新，装潢和设施还不错，只是房间有些油漆味。2。早餐还可以，只是品种不是很多。3。...


In [4]:
## 相对负面一些的话

res_df.tail(10)

,label,review
7756,0,"到半夜竟然没暖气,怎么住啊????!!!!!!!!!!"
7757,0,房间比较差，尤其是洗手间，房间隔音和餐饮服务都不好。
7758,0,不好的，319房间有故臭味。要求换房说满了，我是3月去的。在路上认识了一个上海人，他说他退房...
7759,0,"房间太小,宾馆有租四轮自行车的,很破,骑不动,后来服务员说他们的车都是东方绿舟淘汰的.酒巴装..."
7760,0,我去年的时候去过该酒店，那时候住的是无烟房，感觉还好，这回去住的是高级标准间，进去了房间就有...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
7765,0,说实在的我很失望，之前看了其他人的点评后觉得还可以才去的，结果让我们大跌眼镜。我想这家酒店以...


### 介绍一个好用的断词工具 jieba

* https://github.com/fxsjy/jieba
* 断词的学问博大精深，需要透过大量的语料建模而成
    * 基于前缀词典实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的有向无环图 (DAG)
    * 采用了动态规划查找最大概率路径, 找出基于词频的最大切分组合
    * 对于未登录词，采用了基于汉字成词能力的 HMM 模型，使用了 Viterbi 算法
* 光理解背后的逻辑，得耗费大量的时间，站在巨人的肩膀上也要心怀感激！

In [5]:
!pip install jieba

fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import jieba

#### 试刀一下，怎么好像怪怪的？

In [7]:
' '.join(jieba.lcut(res_df.iloc[9].review))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.810 seconds.
Prefix dict has been built succesfully.


'1 。 酒店 比较 新 ， 装潢 和 设施 还 不错 ， 只是 房间 有些 油漆味 。 2 。 早餐 还 可以 ， 只是 品种 不是 很多 。 3 。 交通 比较 方便 ， 周围 的 小 饭店 比较 多 。'

In [8]:
## 在自然语言处理的领域里，资料清洗是非常重要的一环！垃圾进垃圾出，是无法学习出好的模型的

from src import textutil

In [9]:
def cut_sentence(sentence):   
    return [w for w in jieba.lcut(textutil.clean_sentence(sentence)) if w != ' ']

In [10]:
' '.join(cut_sentence(res_df.iloc[9].review))

'1 酒店 比较 新 装潢 和 设施 还 不错 只是 房间 有些 油漆味 2 早餐 还 可以 只是 品种 不是 很多 3 交通 比较 方便 周围 的 小 饭店 比较 多'

### 支援不同的断词模式

还可以自行加载自定义词库

```python
jieba.load_userdict(file_name)
```

In [11]:
# 全模式
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  

# 精确模式
seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  

# 搜索引擎模式
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  
print("Search Mode: " + "/ ".join(seg_list))

Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
Search Mode: 小明/ 硕士/ 毕业/ 于/ 中国/ 科学/ 学院/ 科学院/ 中国科学院/ 计算/ 计算所/ ，/ 后/ 在/ 日本/ 京都/ 大学/ 日本京都大学/ 深造


In [12]:
all_reviews, all_labels = [], []

res_df = res_df.astype({'review': 'str'})
corpus = []
for idx, row in res_df.iterrows():
    result = cut_sentence(row.review)
    corpus.append(result)
    all_reviews.append(result) 
    all_labels.append(row.label)

In [13]:
corpus[0:3]

[['距离',
  '川沙',
  '公路',
  '较近',
  '但是',
  '公交',
  '指示',
  '不',
  '对',
  '如果',
  '是',
  '蔡陆线',
  '的话',
  '会',
  '非常',
  '麻烦',
  '建议',
  '用',
  '别的',
  '路线',
  '房间',
  '较为简单'],
 ['商务', '大床', '房', '房间', '很大', '床有', '2m', '宽', '整体', '感觉', '经济', '实惠', '不错'],
 ['早餐',
  '太',
  '差',
  '无论',
  '去',
  '多少',
  '人',
  '那边',
  '也',
  '不加',
  '食品',
  '的',
  '酒店',
  '应该',
  '重视',
  '一下',
  '这个',
  '问题',
  '了',
  '房间',
  '本身',
  '很',
  '好']]

### 断词只是第一步，机器怎么知道这些词的意义呢？

试问一下，下列的词汇，意思是相近还是相远的呢？
* 好吃跟难吃
* 美女与野兽
* 天使与恶魔
* 李白跟杜甫

### 强大的工具「词向量」(word2vec) 来了

概念：
* 有些字很常出现在一块，有些却很少，如果大量统计出这种共同出现的机率，就有机会推测出那些词的意义是相近的，而哪些是相远的！

#### 继续站在巨人的肩膀上

In [14]:
!pip install gensim

fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
from gensim.models import word2vec as w2v

#### 参数解释

* size: 词向量的维度
* window: 词的关联区间定义
* min_count: 出现几次的词才加入训练 (当语料非常大，或是资料很脏的时候)
* workers: 用多少个CPU核心训练
* alpha: 学习率(learning rate)
* min_alpha: 学习率最小限制
* iter: 训练回合数

In [16]:
model = w2v.Word2Vec(size=100,  
                     window=5, 
                     min_count=1, 
                     workers=multiprocessing.cpu_count(),
                     alpha=0.025, min_alpha=0.001, iter=50)
model.build_vocab(corpus)
model.train(corpus, epochs=model.iter, total_examples=model.corpus_count)

(21250045, 26058100)

In [17]:
word = '宽敞'
slist = model.wv.most_similar(positive=[word],topn=10)
for word , score in slist :
    print (word)

明亮
布置
舒适
宽敞明亮
整洁
超大
气派
干净
很大
安静


In [18]:
word = ['饭店']
slist = model.wv.most_similar(positive=word,topn=10)
for word , score in slist :
    print (word)

酒店
宾馆
度假村
城市
徽派
旅馆
餐馆
餐饮
山庄
店


In [19]:
os.makedirs('./models', exist_ok=True)
model.save("./models/word2vec.model")

### [试试看] 调整看看 Word2Vec 里面的各式参数，并分享心得

* 重要的参数有哪些呢？
* 修改过参数有什么影响吗？

### 准备朝语意理解迈进拉！

* 先把训练资料准备好

In [20]:
word_list = []
word_counts = model.wv.vectors.shape[0]

for word in model.wv.vocab:
    word_list.append(word)
    
vocab_to_int = {word:model.wv.vocab[word].index for word in word_list}
int_to_vocab = {idx:word for word, idx in vocab_to_int.items()}
int_to_vocab[word_counts] = '###'
vocab_to_int['###'] = word_counts
encoded_reviews = [[vocab_to_int[word] for word in review] for review in all_reviews]

In [21]:
all_reviews[0]

['距离',
 '川沙',
 '公路',
 '较近',
 '但是',
 '公交',
 '指示',
 '不',
 '对',
 '如果',
 '是',
 '蔡陆线',
 '的话',
 '会',
 '非常',
 '麻烦',
 '建议',
 '用',
 '别的',
 '路线',
 '房间',
 '较为简单']

In [22]:
## 看得懂这是什么吗？

encoded_reviews[0]

[423,
 14211,
 3934,
 2542,
 45,
 1551,
 2770,
 13,
 54,
 90,
 3,
 14212,
 215,
 65,
 30,
 864,
 150,
 105,
 639,
 2451,
 5,
 14213]

### 资料清理

* 排除掉可能长度是零的句子
* 透过 `shuffle` 增加排序的乱度

In [23]:
encoded_labels = np.array( [label for idx, label in enumerate(all_labels) if len(encoded_reviews[idx]) > 0] )
encoded_reviews = np.array( [review for review in encoded_reviews if len(review) > 0])

s = np.arange(encoded_labels.shape[0])
np.random.shuffle(s)

encoded_labels = encoded_labels[s]
encoded_reviews = encoded_reviews[s]


print(len(encoded_labels))
print(len(encoded_reviews))

7766
7766


### 句子长度补齐

* 为什么呢？

In [24]:
def pad_text(encoded_reviews, seq_length):
    reviews = []
    for review in encoded_reviews:
        if len(review) >= seq_length:
            reviews.append(review[:seq_length])
        else:
            reviews.append([word_counts]*(seq_length-len(review)) + review)
        
    return np.array(reviews)


padded_reviews = pad_text(encoded_reviews, seq_length = 100)

In [25]:
padded_reviews[0]

array([29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329, 29329,
       29329,   103,    10,    25,  5172,   175,    37,     0,    29,
         558,    79,   145,   236,     0,   299,   108,   196,   256,
       20600,     1,  1475,    98,   289,    15,     4,    14,   123,
          10])

### 切分 `訓練集` 以及 `測試集`

In [26]:
train_ratio = 0.8
test_ratio = 1 - train_ratio
total = padded_reviews.shape[0]
train_cutoff = int(total * train_ratio)

train_x, train_y = padded_reviews[:train_cutoff], encoded_labels[:train_cutoff]
test_x, test_y = padded_reviews[train_cutoff:], encoded_labels[train_cutoff:]

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(6212, 100) (6212,)
(1554, 100) (1554,)


### 开始建模了

* pytorch 的模型有固定的写法
    * 先定义网路层的内容
    * 再定义网路的架构

```python
class MODEL(nn.Module):
    def __init__(self):
        super(MODEL, self).__init__()
        self.lstm = nn.LSTM()
        self.fc = nn.Linear()
        
    def forward(self):
        out = self.lstm()
        out = self.fc(out)
        
        return out
    
```

In [27]:
from torch.utils.data import TensorDataset, DataLoader
import torch.utils.data as Data

In [28]:
class LSTM_SIMPLE(nn.Module):
    def __init__(self, n_vocab, n_embed, n_hidden, n_output, drop_p = 0.5):
        super(LSTM_SIMPLE, self).__init__()
        self.n_vocab = n_vocab     
        self.n_hidden = n_hidden   
        self.embedding = nn.Embedding(n_vocab, n_embed)
        self.lstm = nn.LSTM(n_embed, n_hidden, 1, batch_first = True, 
                            bidirectional=False, dropout = drop_p)
        self.dropout = nn.Dropout(drop_p)
        self.fc = nn.Linear(n_hidden, n_output)
 
    def forward(self, input_words):
        embeds = self.embedding(input_words)    
        lstm_out, _ = self.lstm(embeds, None)         
        lstm_out = self.dropout(lstm_out)
        fc_out = self.fc(lstm_out)[:, -1, :]                     
        
        return fc_out

### 把 `numpy` 资料转换为 PyTorch 的 Tensor 型态

In [29]:
train_x_t = torch.from_numpy(train_x).type(torch.LongTensor)
train_y_t = torch.from_numpy(train_y).type(torch.LongTensor)

test_x_t = torch.from_numpy(test_x).type(torch.LongTensor)
test_y_t = torch.from_numpy(test_y).type(torch.LongTensor)

### 透过 PyTorch DataLoader 模组，做到批次训练的效果

In [30]:
BATCH_SIZE = 32

torch_dataset = Data.TensorDataset(train_x_t, train_y_t)
loader = Data.DataLoader(
    dataset=torch_dataset,  # pytorch TensorDataset 格式
    batch_size=BATCH_SIZE,  # 透过 batch size 设定批次处理
    shuffle=True,           # shuffle 可在每个 epoc 都打乱资料顺序
    num_workers=4,          # 使用几个 CPU 线程做运算
)


In [31]:
n_vocab = len(vocab_to_int)
n_embed = 100
n_hidden = 64
n_output = 2

### 看一下网路架构

In [32]:
net_s = LSTM_SIMPLE(n_vocab, n_embed, n_hidden, n_output)
print(net_s)  

optimizer = torch.optim.Adam(net_s.parameters(), lr=0.02)
loss_func = torch.nn.CrossEntropyLoss()  

plt.ion()  

LSTM_SIMPLE(
  (embedding): Embedding(29330, 100)
  (lstm): LSTM(100, 64, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)


In [33]:
for epoch in range(5):   
    for step, (batch_x, batch_y) in enumerate(loader):  
        out = net_s(batch_x)
        loss = loss_func(out, batch_y)
        
        optimizer.zero_grad()   # 清空 gradients
        loss.backward()         # 倒传导参数训练
        optimizer.step()        # 调整参数
        
        if step % 50 == 0:      # 每 50 步印一个结果
            test_output = net_s(test_x_t)                   
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 0.7190 | test accuracy: 0.64
Epoch:  0 | train loss: 0.6759 | test accuracy: 0.68
Epoch:  0 | train loss: 0.6363 | test accuracy: 0.71
Epoch:  0 | train loss: 0.7152 | test accuracy: 0.71
Epoch:  1 | train loss: 0.4450 | test accuracy: 0.80
Epoch:  1 | train loss: 0.5015 | test accuracy: 0.79
Epoch:  1 | train loss: 0.3811 | test accuracy: 0.77
Epoch:  1 | train loss: 0.5694 | test accuracy: 0.80
Epoch:  2 | train loss: 0.2369 | test accuracy: 0.78
Epoch:  2 | train loss: 0.1497 | test accuracy: 0.82
Epoch:  2 | train loss: 0.4146 | test accuracy: 0.80
Epoch:  2 | train loss: 0.3402 | test accuracy: 0.82
Epoch:  3 | train loss: 0.2659 | test accuracy: 0.82
Epoch:  3 | train loss: 0.1055 | test accuracy: 0.82
Epoch:  3 | train loss: 0.5010 | test accuracy: 0.82
Epoch:  3 | train loss: 0.2424 | test accuracy: 0.82
Epoch:  4 | train loss: 0.1083 | test accuracy: 0.82
Epoch:  4 | train loss: 0.1214 | test accuracy: 0.82
Epoch:  4 | train loss: 0.1001 | test accuracy

### 还记得 transfer learning 的概念吗？

* 把词向量当成预设的 embeddings layer
* 学习速度会加快，而且效果更好

#### 记得配给  `###`  一个预设零向量

In [34]:
weights = torch.FloatTensor(np.vstack([model.wv.vectors, np.zeros(100)]))

In [35]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False  ## 把 embedding 层的参数固定住了

    return emb_layer, num_embeddings, embedding_dim

In [36]:
class LSTM_WV(nn.Module):
    def __init__(self, weights, n_hidden, n_output, drop_p = 0.5):
        super(LSTM_WV, self).__init__()
        self.n_vocab = n_vocab     
        self.n_hidden = n_hidden   
        self.embedding, num_embeddings, n_embed = create_emb_layer(weights)
        self.lstm = nn.LSTM(n_embed, n_hidden, 1, batch_first = True, 
                            bidirectional=True, dropout = drop_p)
        self.dropout = nn.Dropout(drop_p)
        self.fc = nn.Linear(n_hidden*2, n_output)
 
    def forward(self, input_words):
        embeds = self.embedding(input_words)    
        lstm_out, _ = self.lstm(embeds, None)         
        lstm_out = self.dropout(lstm_out)
        fc_out = self.fc(lstm_out)[:, -1, :]                     
        
        return fc_out
    
    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(2, bsz, self.n_hidden),
                weight.new_zeros(2, bsz, self.n_hidden))


In [37]:
net_wv = LSTM_WV(weights, n_hidden, n_output)
print(net_wv)  

optimizer = torch.optim.Adam(net_wv.parameters(), lr=0.02)
loss_func = torch.nn.CrossEntropyLoss()  

plt.ion()   

LSTM_WV(
  (embedding): Embedding(29330, 100)
  (lstm): LSTM(100, 64, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


### 开始训练

In [38]:
cur_loss = 10000
cur_acc = 0

for epoch in range(10):   
    for step, (batch_x, batch_y) in enumerate(loader):  
        net_wv.init_hidden(len(batch_y))
        out = net_wv(batch_x)
        loss = loss_func(out, batch_y)
        
        optimizer.zero_grad()   
        loss.backward()         
        optimizer.step()        
        
        if step % 50 == 0:
            net_wv.init_hidden(len(batch_y))
            test_output = net_wv(test_x_t)                   
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            if accuracy >= cur_acc and loss.item() <= cur_loss:   ### 只要模型效果优化了，就重新储存
                torch.save(net_wv.state_dict(), f'./models/lstm_{epoch}_{step}.param')
                cur_acc = accuracy
                cur_loss = loss.item()
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            
            

Epoch:  0 | train loss: 0.6818 | test accuracy: 0.67
Epoch:  0 | train loss: 0.6104 | test accuracy: 0.71
Epoch:  0 | train loss: 0.5192 | test accuracy: 0.76
Epoch:  0 | train loss: 0.6178 | test accuracy: 0.77
Epoch:  1 | train loss: 0.4363 | test accuracy: 0.79
Epoch:  1 | train loss: 0.3275 | test accuracy: 0.79
Epoch:  1 | train loss: 0.4032 | test accuracy: 0.79
Epoch:  1 | train loss: 0.2616 | test accuracy: 0.80
Epoch:  2 | train loss: 0.2454 | test accuracy: 0.79
Epoch:  2 | train loss: 0.5057 | test accuracy: 0.81
Epoch:  2 | train loss: 0.2937 | test accuracy: 0.80
Epoch:  2 | train loss: 0.2072 | test accuracy: 0.81
Epoch:  3 | train loss: 0.0336 | test accuracy: 0.81
Epoch:  3 | train loss: 0.0364 | test accuracy: 0.81
Epoch:  3 | train loss: 0.0557 | test accuracy: 0.81
Epoch:  3 | train loss: 0.2125 | test accuracy: 0.82
Epoch:  4 | train loss: 0.1494 | test accuracy: 0.80
Epoch:  4 | train loss: 0.0683 | test accuracy: 0.81
Epoch:  4 | train loss: 0.2154 | test accuracy

In [39]:
predictor = LSTM_WV(weights, n_hidden, n_output)

In [40]:
predictor.load_state_dict(torch.load('./models/lstm_4_150.param'))

<All keys matched successfully>

In [41]:
def predict(review, seq_length = 100):
    words = cut_sentence(review)
    encoded_words = [vocab_to_int[word] for word in words]
    padded_words = pad_text([encoded_words], seq_length)
    padded_words = torch.from_numpy(padded_words)
    output = predictor(padded_words)
    pred = torch.max(output, 1)[1].data.numpy()

    msg = "顾客很开心喔！" if pred == 1 else "服务要再加强了"
    
    return msg

In [42]:
review1 = "闹中取静的一个地方，在窗前能看到不错的风景。"
review2 = "位置不错,在市中心.周围吃饭等很方便.房间一如既往的干净"
review3 = "这个宾馆好像是属于海军的南海舰队，地理位置也很好，靠近省委火车站，离黄兴步行街也就三站地，值得入住"
review4 = "房间比较差，尤其是洗手间，房间隔音和餐饮服务都不好"
review5 = "硬件差：房间设施简单、陈旧，房内有较重的异味，半夜后中央空调“罢工”。服务不好，房间差"
review6 = "真是有够烂到爆，再也不想住了，太夸张，很差非常差"
review7 = "这家餐厅真的很棒 很舒服 服务又好 下次一定会再来"

print(predict(review1)) 
print(predict(review2))
print(predict(review3))
print(predict(review4))
print(predict(review5))
print(predict(review6))
print(predict(review7))

顾客很开心喔！
顾客很开心喔！
顾客很开心喔！
服务要再加强了
服务要再加强了
服务要再加强了
顾客很开心喔！
